# Resnet50

In [ ]:
import numpy as np
from sklearn.datasets import load_sample_image

# Load sample images
# Pixel intesities vary between 0-255, scale to 0-1
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

In [ ]:
# resize image -> resnet expects 224X224
import tensorflow as tf 

images_resized = tf.image.resize(images,[224,224])

In [ ]:
# our images are in scale 0-1; preprocess function assumes it is from 0-255, so multiply back
inputs = keras.applications.resnet50.preprocess_input(images_resized *255)

In [ ]:
Y_proba = model.predict(inputs)


In [ ]:
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print("Image #{}".format(image_index))
    for class_id, name, y_proba in top_K[image_index]:
        print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba *100))
        print()


In [ ]:
!pip uninstall  tensorflow_datasets



# Transfer Learning

In [12]:
# Use Xception 
import tensorflow_datasets as tfds
import tensorflow as tf 


tfds.disable_progress_bar()

(train_set, test_set, valid_set), info = tfds.load("tf_flowers",
                          split=[
                                tfds.Split.TRAIN.subsplit(tfds.percent[:70]),
                                tfds.Split.TRAIN.subsplit(tfds.percent[70:85]),
                                tfds.Split.TRAIN.subsplit(tfds.percent[85:]),
                              
                              
                          ],
                          as_supervised=True,with_info=True)

#info -> setting with_info = True

dataset_size = info.splits["train"].num_examples # 3670
class_names = info.features["label"].names # ["dandelion", "daisy", ...]
n_classes = info.features["label"].num_classes # 5

In [13]:
def preprocess(image, label):
        resized_image = tf.image.resize(image, [224, 224])
        final_image = keras.applications.xception.preprocess_input(resized_image)
        return final_image, label


In [14]:
import tensorflow.keras as keras 

batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [15]:
# Remove the top layer ie the global average pooling layer + dense output layer
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# add new gobal layer + Dense output layer

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)

# keras model outputs = output implies using the new model
model = keras.Model(inputs=base_model.input, outputs=output)


In [16]:
# Freeze the base layers in the initial layers of training

for layer in base_model.layers:
    layer.trainable = False

In [19]:
import tensorflow as tf

optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(train_set, epochs=2, validation_data=valid_set)

Epoch 1/2
81/81 [==============================] - 217s 3s/step - loss: 1.6161 - accuracy: 0.7838 - val_loss: 1.0758 - val_accuracy: 0.8574
Epoch 2/2
38/81 [=============>................] - ETA: 1:41 - loss: 0.4977 - accuracy: 0.8964Collecting tensorflow-gpu
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\91760\\anaconda3\\envs\\tensorflow_gpu\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



81/81 [==============================] - 230s 3s/step - loss: 0.5301 - accuracy: 0.8992 - val_loss: 0.9113 - val_accuracy: 0.8722


In [23]:
Y_proba = model.predict(test_set)

top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
        print("Image #{}".format(image_index))
        for class_id, name, y_proba in top_K[image_index]:
            print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba *100))
            print()


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (540, 5)

In [26]:
train_set['label']

TypeError: 'PrefetchDataset' object is not subscriptable